# Data Export

This notebook outlines browsing of the database and use of the `pipeline.export` module to export recording data.

The data format is a custom format used internally by project collaborators to support non-datajoint code, and supports export of processed per-probe electrophysiology data along with the experimental events pertaining to that data.

If a session is known, the export capability is also available from the `mapshell.py` script; this usage will be documented below.

In [3]:
from pipeline import lab
from pipeline import experiment
from pipeline import ephys
from pipeline import export

Connecting root@localhost:3306


## Find Session / Probe Insertion

In [4]:
experiment.Session()

subject_id institution 6 digit animal ID,session session number,session_date,session_time,username,rig
412330,1,2018-07-16,15:46:24,unknown,RRig
412753,1,2018-07-10,14:19:59,unknown,RRig
413950,1,2018-10-22,10:47:29,unknown,RRig
432572,1,2018-11-25,19:33:26,unknown,RRig
435884,1,2018-12-07,14:51:39,unknown,RRig
456772,1,2019-11-23,14:51:27,unknown,RRig2
460436,1,2020-01-10,14:27:17,unknown,RRig2


In [3]:
skey = {'subject_id': 435884, 'session': 1, 'session_date': '2018-12-07'}

In [4]:
ephys.ProbeInsertion & skey

subject_id institution 6 digit animal ID,session session number,insertion_number,probe unique identifier for this model of probe (e.g. part number),electrode_config_name user friendly name
435884,1,1,15131808323,npx_first384


In [5]:
ekey = {**skey, 'insertion_number': 1}

## Export Data

The `export.export_recording` function is used to export data. Online help is available as follows

In [5]:
help(export.export_recording)

Help on function export_recording in module pipeline.export:

export_recording(insert_keys, output_dir='./', filename=None)
    Export a 'recording' (or a list of recording) (probe specific data + related events) to a file.
    
    Parameters:
    
      - insert_keys: one or a list of ephys.ProbeInsertion.primary_key
        currently: {'subject_id', 'session', 'insertion_number'})
    
      - output_dir: directory to save the file at (default to be the current working directory)
    
      - filename: an optional output file path string. If not provided,
        filename will be autogenerated using the 'mkfilename'
        function.
        Note: if exporting a list of probe keys, filename will be auto-generated



In [6]:
help(export.mkfilename)

Help on function mkfilename in module pipeline.export:

mkfilename(insert_key)
    create a filename for the given insertion key.
    filename will be of the format map-export_h2o_YYYYMMDD_HHMMSS_SN_PN.mat
    
    where:
    
      - h2o: water restriction number
      - YYYYMMDD_HHMMSS: session recording datetime
      - SN: session number for this subject
      - PN: probe number for this session



In [8]:
export.export_recording(ekey)

exporting {'subject_id': 435884, 'session': 1, 'session_date': '2018-12-07', 'insertion_number': 1} to map-export_dl59_2018-12-07_1_1.mat
fetching spike/behavior data
reshaping/processing for export
saving to map-export_dl59_2018-12-07_1_1.mat


## Export Data (pipeline.shell - interactive)

The `pipeline.shell` module wrapps the export_recording fuction - usage is as above:

```
from pipeline import shell
shell.export_recording(ekey)
```

## Export Data (mapshell.py)

To use the `export-recording` feature of the `mapshell.py` convenience script, pass in a quoted key and optional filename:


```
$ mapshell.py export-recording "{'subject_id': 435884, 'session': 1, 'session_date': '2018-12-07', 'insertion_number': 1}"
```

### Note: `export.export_recording()` can also work on a list of `ekey` as input, for batch export